In [1]:
include("setup.jl");

  Activating project at `~/docs/JuliaCon/2023/HPC`
[ Info: Precompiling Dagger [d58978e5-989f-55fb-8d15-ea34adc7bf54]
[ Info: Precompiling DaggerGPU [68e73e28-2238-4d5a-bf97-e5d4aa3c4be2]


![](https://github.com/JuliaParallel/Dagger.jl/raw/master/docs/logo.jpg)

## Dagger for HPC

#### Written by: Julian Samaroo, Research Software Engineer at MIT's JuliaLab, maintainer of Dagger.jl

In [2]:
using Dagger

## What is Dagger?
- A Julia library that makes parallel programming portable, performant, and productive
- Unifies Julia's multithreading, multiprocessing (distributed), and GPU computing capabilities
- Easy to write code on a laptop and deploy it at scale (supercomputer, cloud, etc.)

### Ecosystem problems:
1. Multithreading, distributed, and GPU computing don't compose
2. Efficient task scheduling and data movement is hard to do manually
3. N-to-M problem of abstractions to parallelism strategies
4. Different end-user hardware configurations

### Dagger's solutions:
1. Unified task and data movement concepts
2. Heterogeneous scheduler using machine learning
3. High-level APIs all call the same task API underneath
4. Hardware awareness and flexible processor abstractions

## Enough talk! This is JuliaCon, show me the code!

### Distributed Arrays

Let's look at all of Dagger's various interfaces, starting with the distributed array interface, the `DArray`:

In [3]:
A = rand(64, 64)

# Let's distribute A across our workers by partitioning in 16 x 16 blocks
DA = distribute(A, Blocks(16, 16))

# Do an operation on it:
DB = map(x->x*2, DA)

# And get the result back as an Array:
collect(DB)

64×64 Matrix{Float64}:
 1.42442   1.75499     0.512983  …  1.54363   0.10699   1.74991
 0.187488  1.04904     0.535902     0.991907  1.95852   1.46149
 0.504441  0.608903    1.24177      0.369546  1.86473   0.610309
 0.425605  0.308427    0.551054     0.604314  1.58523   0.819905
 1.56785   0.943423    1.23804      0.272528  0.921236  1.59822
 1.29823   1.31296     1.92522   …  0.497407  0.372783  0.309022
 0.121636  0.525815    0.345753     0.304693  1.63458   0.844098
 0.631788  1.41564     0.772955     1.40852   0.672284  0.0138266
 0.253049  1.78613     0.579613     0.930276  1.31191   0.141314
 0.444186  1.26853     1.59303      0.96387   1.40387   0.399772
 1.97589   0.675521    0.726675  …  0.724015  1.67095   1.86371
 0.129561  0.085523    1.292        0.94088   1.56437   1.79553
 0.496854  1.14083     1.15055      1.71245   1.67252   0.400495
 ⋮                               ⋱                      
 0.318141  1.2417      1.86648      1.73593   0.688311  0.30338
 0.543354  0.38

In [4]:
# Or allocate an array directly:
collect(rand(Blocks(16, 16), 64, 64))

64×64 Matrix{Float64}:
 0.658126   0.283695   0.796985   …  0.749473  0.811277   0.0527573
 0.203491   0.294815   0.581946      0.209931  0.952948   0.34408
 0.604561   0.334882   0.787896      0.500641  0.969723   0.746681
 0.198857   0.670821   0.128755      0.856966  0.644631   0.251717
 0.603761   0.0260362  0.939373      0.832957  0.39569    0.530448
 0.566787   0.663621   0.642383   …  0.958785  0.943038   0.918318
 0.31308    0.331933   0.319969      0.44801   0.30823    0.446051
 0.0946     0.364543   0.58955       0.470084  0.289969   0.916626
 0.833488   0.610157   0.530583      0.731231  0.598206   0.274763
 0.931132   0.633184   0.448787      0.969361  0.590173   0.705504
 0.869467   0.104607   0.652301   …  0.652722  0.418679   0.396207
 0.858183   0.488585   0.934774      0.252574  0.755862   0.371537
 0.0574044  0.705903   0.0725693     0.60596   0.0762379  0.545298
 ⋮                                ⋱                       
 0.431993   0.844963   0.257325      0.824322  

In [5]:
# Reductions and broadcast work too!
@show sum(DA)
collect(DA .* 2 .+ DB)

sum(DA) = 2031.5587026509343


64×64 Matrix{Float64}:
 2.84885   3.50999     1.02597   3.34773    …  3.08727   0.213979  3.49983
 0.374976  2.09808     1.0718    0.578355      1.98381   3.91705   2.92298
 1.00888   1.21781     2.48354   1.06243       0.739091  3.72946   1.22062
 0.85121   0.616854    1.10211   1.22413       1.20863   3.17046   1.63981
 3.1357    1.88685     2.47609   0.225156      0.545056  1.84247   3.19644
 2.59647   2.62592     3.85044   3.95635    …  0.994813  0.745566  0.618044
 0.243271  1.05163     0.691505  3.28188       0.609386  3.26917   1.6882
 1.26358   2.83128     1.54591   2.02111       2.81703   1.34457   0.0276533
 0.506098  3.57226     1.15923   0.668858      1.86055   2.62382   0.282628
 0.888372  2.53705     3.18606   3.48046       1.92774   2.80775   0.799543
 3.95178   1.35104     1.45335   3.72815    …  1.44803   3.34189   3.72742
 0.259122  0.171046    2.584     2.61396       1.88176   3.12874   3.59105
 0.993709  2.28166     2.30109   3.6471        3.4249    3.34505   0.8009

Where's the proof that it's distributed?

In [6]:
# We can easily see which worker we're running on:
collect(map(x->Distributed.myid(), DB))

64×64 Matrix{Int64}:
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     3  3  3  3  3  3  3  3  3  3  3  3
 2  2  2  2  2  2  2  2  2  2  2  2  2     

Dagger also automates multithreading:

In [7]:
# And we can check which thread we're on too:
collect(map(x->Threads.threadid(), DB))

64×64 Matrix{Int64}:
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2  …  1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  2  2  2  2  2  2  2  2     

#### Supercomputer parallelism: MPI

Amazingly, thanks to our Google Summer of Code student Felipe Tome, the `DArray` also supports MPI:

In [ ]:
# We won't run this because we aren't running under MPI
# Try it on your favorite supercomputer :)

using DaggerMPI, MPI

comm = MPI.COMM_WORLD
nranks = MPI.Comm_size(comm)

# Automatically partition array across MPI ranks
DA = rand(MPIBlocks(64, nothing), 64, 64)
# Same here!
DB = distribute(rand(64, 64), MPIBlocks(64, nothing))

# Array operations work fine
# (They also automatically use MPI!)
DC = map(x->x+1, DA)
DD = DB .+ DC
collect(DD) # Exactly what you'd expect

### Distributed Tables

Let's see other abstractions; the DTables.jl package exposes distributed tables:

In [15]:
using DTables, DataFrames

df = DataFrame(a=rand(1:100), b=rand('a':'z', 100))
dtbl = DTable(df, 20) # Partition into sub-tables of 20 rows each

DTable with 5 partitions
Tabletype: DataFrame

Like the `DArray`, the `DTable` is split into partitions:

In [17]:
@show fetch(dtbl.chunks[2]) fetch(dtbl.chunks[4])

fetch(dtbl.chunks[2]) = 20×2 DataFrame
 Row │ a      b
     │ Int64  Char
─────┼─────────────
   1 │    94  y
   2 │    94  u
   3 │    94  h
   4 │    94  w
   5 │    94  b
   6 │    94  x
   7 │    94  a
   8 │    94  h
   9 │    94  e
  10 │    94  o
  11 │    94  v
  12 │    94  v
  13 │    94  q
  14 │    94  y
  15 │    94  b
  16 │    94  u
  17 │    94  r
  18 │    94  r
  19 │    94  g
  20 │    94  k
fetch(dtbl.chunks[4]) = 20×2 DataFrame
 Row │ a      b
     │ Int64  Char
─────┼─────────────
   1 │    94  o
   2 │    94  x
   3 │    94  o
   4 │    94  l
   5 │    94  x
   6 │    94  r
   7 │    94  j
   8 │    94  q
   9 │    94  u
  10 │    94  c
  11 │    94  r
  12 │    94  l
  13 │    94  m
  14 │    94  q
  15 │    94  j
  16 │    94  k
  17 │    94  j
  18 │    94  n
  19 │    94  j
  20 │    94  m


Row,a,b
,Int64,Char
1,94,o
2,94,x
3,94,o
4,94,l
5,94,x
6,94,r
7,94,j
8,94,q
9,94,u


And it's quite easy to show that operations are distributed+multithreaded:

In [18]:
map(row->merge(row, (;wid=Distributed.myid(), tid=Threads.threadid())), dtbl) |> DataFrame

Row,a,b,wid,tid
,Int64,Char,Int64,Int64
1,94,y,3,2
2,94,w,3,2
3,94,f,3,2
4,94,h,3,2
5,94,y,3,2
6,94,w,3,2
7,94,j,3,2
8,94,m,3,2
9,94,l,3,2


#### How do these abstractions work?

- All computations done in Dagger tasks
- All data stored in Dagger `Chunk`s
- Single unified scheduler handles all the details

This gives us free features like distributed+multithreaded computing, out-of-core computing, and more!

### Distributed Loops and Transducers

These simple core features also make it easy to build other abstractions, like accelerated loops and transducers:

In [23]:
using FoldsDagger, FLoops, Referenceables

# Note: We're locking this array and computations to worker 1
# because of some unresolved distributed issues with FoldsDagger
Dagger.with_options(;scope=Dagger.scope(worker=1)) do
    DC = rand(Blocks(4), 16)

    @show collect(DC)[1:4]

    @floop DaggerEx() for x in referenceable(DC)
        x[] += 42.0
    end

    @show collect(DC)[1:4]
end;

(collect(DC))[1:4] = [0.6700156037773604, 0.8285620101463849, 0.43402200373501265, 0.9041125621838035]
(collect(DC))[1:4] = [42.67001560377736, 42.828562010146385, 42.43402200373501, 42.904112562183805]


In [5]:
using Folds, Transducers

Dagger.with_options(;scope=Dagger.scope(worker=1)) do
    DC = rand(Blocks(4), 16)

    #@show Folds.sum(DC)
    # For comparison:
    @show sum(DC) sum(collect(DC))
    
    DD = DC |> Filter(x->x < 0.5)
    @show collect(DD)
end;

sum(DC) = 7.603597375606933
sum(collect(DC)) = 7.603597375606933


[ Info: Failed with CartesianIndex(1,) 1 1


LoadError: MethodError: no method matching getindex(::CartesianIndex{1}, ::UnitRange{Int64})

[0mClosest candidates are:
[0m  getindex(::CartesianIndex, [91m::Integer[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmultidimensional.jl:90[24m[39m


### Distributed SPMD Kernels

With DaggerGPU and KernelAbstractions, executing SPMD kernels is super easy:

In [1]:
using DaggerGPU, KernelAbstractions
import DaggerGPU: Kernel

@kernel function vinc!(O, A)
    idx = @index(Global, Linear)
    O[idx] = A[idx] + 1
end

@kernel function vadd!(C, A, B)
    idx = @index(Global, Linear)
    C[idx] = A[idx] + B[idx]
end

# Start with CPU arrays, locked to this worker
A = Dagger.@mutable rand(Float32, 64)
O = Dagger.@mutable zeros(Float32, 64)

# Launch the kernel in a task and wait for it to finish
wait(Dagger.@spawn Kernel(vinc!)(O, A))

fetch(O) == fetch(A) .+ 1

This worked! But it only used the CPU - boring! Thankfully, GPU acceleration is trivial:

In [ ]:
using AMDGPU

# This time we use AMDGPU's ROCArrays
A = Dagger.@mutable AMDGPU.rand(Float32, 64)
O = Dagger.@mutable AMDGPU.zeros(Float32, 64)

# Specify a "scope" of execution on only the first ROCm (AMD) GPU
gpu_scope = Dagger.scope(rocm_gpu=1)

# or, for using any GPU, do:
# gpu_scope = Dagger.ProcessorTypeScope(DaggerGPU.ROCArrayDeviceProc)
# (This will become easier to specify in the future!)

# Launch the kernel on the first AMD GPU
Dagger.with_options(;scope=gpu_scope) do
    wait(Dagger.@spawn Kernel(vinc!)(O, A))
end

fetch(O) == fetch(A) .+ 1

Ok, that's fine, but this is only one kernel; what if I need to launch a bunch in sequence? No problem - `spawn_sequential` to the rescue!

In [ ]:
A = Dagger.@mutable AMDGPU.rand(Float32, 64)
B = Dagger.@mutable AMDGPU.rand(Float32, 64)
C = Dagger.@mutable AMDGPU.zeros(Float32, 64)
O = Dagger.@mutable AMDGPU.zeros(Float32, 64)

# Launch two kernels in sequential order
Dagger.with_options(;scope=gpu_scope) do
    wait(Dagger.spawn_sequential() do
        Dagger.@spawn Kernel(vadd!)(C, A, B)
        Dagger.@spawn Kernel(vinc!)(O, C)
    end)
end

fetch(O) == fetch(A) .+ fetch(B) .+ 1

Above, even though Dagger normally only orders by function arguments (requiring annoying tricks to make work), `spawn_sequential` forced sequential task ordering for each submitted task.

What about if I need parallel kernels? No problem, just use `spawn_bulk` within:

In [ ]:
Z1 = Dagger.@mutable AMDGPU.rand(Float32, 64)
Z2 = Dagger.@mutable AMDGPU.rand(Float32, 64)
A = Dagger.@mutable AMDGPU.zeros(Float32, 64)
B = Dagger.@mutable AMDGPU.zeros(Float32, 64)
C = Dagger.@mutable AMDGPU.zeros(Float32, 64)

Dagger.with_options(;scope=gpu_scope) do
    wait(Dagger.spawn_sequential() do
        # Launch two kernels in parallel
        Dagger.spawn_bulk() do
            Dagger.@spawn Kernel(vinc!)(A, Z1)
            Dagger.@spawn Kernel(vinc!)(B, Z2)
        end

        # Once those are done, run this kernel
        Dagger.@spawn Kernel(vadd!)(C, A, B)
    end)
end

fetch(C) == fetch(Z1) .+ 1 .+ fetch(Z2) .+ 1

Cool - `spawn_bulk` let us get some controlled parallelism even when we were in a sequential region!

### GPU Array Programming

Kernels aren't the only option for GPUs - array programming works just fine:

In [ ]:
# Start with a CPU array (this time, not locked to a worker)...
A = rand(Float32, 64)

# ... and do some array ops, but all on the GPU!
C = Dagger.with_options(;scope=gpu_scope) do
    # No need to manually convert to a ROCArray
    # Dagger converts and moves data for you
    B = Dagger.@spawn A .+ 3
    Dagger.@spawn map(x->x/2, B)
end

# Let's finish with reducing on the CPU
# Again, no need to convert manually!
fetch(Dagger.@spawn sum(C))

### In summary:

#### Key Points

- Many computational abstractions (array, table, task, kernel)
- Integrates with key ecosystem packages (MPI.jl, KernelAbstractions.jl, CUDA/AMDGPU/etc.)
- Parallelism is automatic
- Change parallelism strategy from top level

#### More work to be done (help wanted!)

- DArray + KernelAbstractions integration
- Distributed graphs implementation (social networks, security research, etc.)
- Intel GPU, GraphCore IPU, etc. support in DaggerGPU

#### What can you do?

- Use Dagger to solve your computational problems
- Report any bugs or performance issues
- Add documentation and examples
- Add integrations between Dagger and other libraries